### Choosing exercise routine
- choosing an exercise routine that burns the max number of calories. A function has been defined, based on the input of pushups and miles ran, will print the number of calories burned, time used, and if that time is greater than 10 minutes.

In [16]:
from pulp import *

In [15]:
# Initialize Class
model = LpProblem("Maximize calories burnt", LpMaximize)

# Define Decision Variables
num_pushup = LpVariable('Pushups', lowBound=0, upBound=None, cat='Integer')
num_miles_ran = LpVariable('Running', lowBound=0, upBound=None, cat='Integer')

# Define Objective Function
model += 3.0 * num_pushup + 130.0 * num_miles_ran

# Define Constraints
model += 0.2 * num_pushup + 10.0 * num_miles_ran <= 10.0
model += num_pushup >= 0
model += num_miles_ran >= 0

# Solve Model
model.solve()
print("{} pushups".format(num_pushup.varValue))
print("{} running".format(num_miles_ran.varValue))

50.0 pushups
0.0 running


In [1]:
def lp(num_pushup, num_miles_ran):
    """This function prints the number of calories burned, time used, and if that time is greater than 10 minutes"""
    obj = 3.0 * num_pushup + 130.0 * num_miles_ran
    print("Calories Burned: {0}".format(obj))
    min = 0.2 * num_pushup + 10.0 * num_miles_ran
    print("Time Used: {0}".format(min))
    print("Used too much time: {0}".format(min>10.0))

In [2]:
num_pushup = 50
num_miles_ran = 0
lp(num_pushup, num_miles_ran)

Calories Burned: 150.0
Time Used: 10.0
Used too much time: False


In [3]:
num_pushup = 0
num_miles_ran = 1
lp(num_pushup, num_miles_ran)

Calories Burned: 130.0
Time Used: 10.0
Used too much time: False


In [4]:
num_pushup = 30
num_miles_ran = 0.5
lp(num_pushup, num_miles_ran)

Calories Burned: 155.0
Time Used: 11.0
Used too much time: True


### Simple resource scheduling exercise
- In this exercise we are planning the production at a glass manufacturer. This manufacturer only produces wine and beer glasses:
- there is a maximum production capacity of 60 hours
- each batch of wine and beer glasses takes 6 and 5 hours respectively
- the warehouse has a maximum capacity of 150 rack spaces
- each batch of the wine and beer glasses takes 10 and 20 spaces respectively
- the production equipment can only make full batches, no partial batches

- Also, we only have orders for 6 batches of wine glasses. Therefore, we do not want to produce more than this. Each batch of the wine glasses earns a profit of `$ 5` and the beer `$ 4.5`
- The objective is to maximize the profit for the manufacturer.

In [7]:
from pulp import *

In [8]:
# Initialize Class
model = LpProblem("Maximize Glass Co. Profits", LpMaximize)

# Define Decision Variables
wine = LpVariable('Wine', lowBound=0, upBound=None, cat='Integer')
beer = LpVariable('Beer', lowBound=0, upBound=None, cat='Integer')

# Define Objective Function
model += 5 * wine + 4.5 * beer

# Define Constraints
model += 6 * wine + 5 * beer <= 60
model += 10 * wine + 20 * beer <= 150
model += wine <= 6

# Solve Model
model.solve()
print("Produce {} batches of wine glasses".format(wine.varValue))
print("Produce {} batches of beer glasses".format(beer.varValue))

C:\Users\E1366134\Anaconda3\lib\site-packages\pulp\pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Produce 6.0 batches of wine glasses
Produce 4.0 batches of beer glasses


##### Using `LpMaximize()` makes sense because we are trying to maximize profits. It does not make sense to set an upper bound on the number of wine or beer glasses produced: the more the better. However, setting a lower bound is important because producing negative glasses is impossible. Also, selecting the variable to be an integer makes sense because we can not make half batches. Adding the last constraint on wine based on current orders is important to add to avoid producing too many wine glasses.